# word2vec 구현하기

## 추론 기반 기법과 신경망  
-확률 기반으로 단어를 벡터로 표현하는것에는 한계가 있음.  
-예를 들어 svd를 사용한다고 하더라고 100만 단어 종류가 있을 경우 계산량이 상당히 많이 들어감.(n^3)  
-따라서 미니배치 방식의 딥러닝 방식이라면 100만 단어라도 충분히 처리가 가능함.

## 추론기반 기법?  
-추론 기반 기법이란, 간단히 말해서 문맥을 통해서 어떤 단어가 와야할지를 예측하는 방법.

## 전처리 - 단어
-신경망에서는 단어 문자 그대로를 처리할수 없으므로 단어를 처리할수 있는 형태를 취해야함.  
-그게 흔히 많이 사용하는 방법인 one-hot 표현 방식임.  
-c는 임의의 원핫으로 표현한 벡터, 여기에 가중치(웨이트)W를 곱해서 단일 히든 레이어를 구성.

In [3]:
import numpy as np

c = np.array([1,0,0,0,0,0,0])
W = np.random.randn(7,3)
h = np.matmul(c, W)
print(h)

[ 0.84753331  0.31089763  0.78646923]


앞에서 사용했던 레이어를 다시 사용하자.

In [5]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grad = [np.zeros_like(W)]
        self.x =None
        
    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out
        
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        
        ## numpy deep copy.
        # 완전히 같지는 않지만 C언어에서 포인터의 느낌. 생략기호를 넣으면 값이 완전히 바뀌고,
        # 일반적으로 하듯이 대입을 하면 메모리를 가르키는 위치만 변경.
        self.grads[0][...] = dW
        return dx

In [9]:
import sys
sys.path.append('..')
import numpy as np

c = np.array([1, 0, 0, 0, 0, 0, 0])
W = np.random.randn(7,3)
layer = MatMul(W)

h = layer.forward(c)
print(h)

[-0.02087071 -0.98057648 -1.63714674]


# CBow(continuous bag of words)형태의 word2vec  
-두개의 입력층을 가지고 하나의 은닉층을 거쳐 출력을 하는 형태로 구현 

## 먼저는 인퍼런스 부분만 구현해봅시다.

In [16]:
import sys
sys.path.append('..')
import numpy as np

#두 개의 임의의 입력 벡터 선언
c0 = np.array([1, 0, 0, 0, 0, 0, 0])
c1 = np.array([0, 0, 1, 0, 0, 0, 0])

W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[ 6.60864335  2.10805258 -1.60691172 -3.35894147 -4.6980309   3.07601584
 -1.28832815]


-여기서 생각할만한 것은 word2vec을 하는 목표가 단어의 분산표현임.  
-하지만 W_in W_out둘다 단어의 분산표현이라고 할수 있을텐데 어떤것을 사용할지가 관건. 주로 W_in을 사용하는게 대세라고 함.

## 이제는 역전파를 적용해서 학습까지...  
-학습에는 cross entropy & softmax를 사용합니다.  
-먼저는 앞에서 구현했던 전처리 코드를 다시 사용합니다.

In [17]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word


In [19]:
import sys
sys.path.append('..')

text = 'You say goodbye and i say hello'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello'}


-이 코드를 사용하면 문자을 단어로 쪼개고, 각 단어들에게 번호, 라벨을 부여합니다.  
-다음으로는 cbow의 학습 데이터 형태를 위해 데이터 전처리 준비를 합니다.  
-cbow는 문맥이라고 하는 즉, 맞추고자 하는 단어를 기준으로 양옆과 같은 주위에 있는 단어를 입력으로 합니다.

In [18]:
def create_contexts_target(corpus, window_size=1):

    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)

    return np.array(contexts), np.array(target)


In [23]:
contexts, target = create_contexts_target(corpus, window_size=1)

print(contexts)
print(target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]]
[1 2 3 4 1]


-이제 데이터를 사용하기 위해서 라벨 값들을 원핫 백터로 표현해주는 함수를 구현하자.


In [24]:
def convert_one_hot(corpus, vocab_size):

    N = corpus.shape[0]

    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1

    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1

    return one_hot


In [26]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None
        self.t = None  

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx

In [27]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]

    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [32]:
import sys
sys.path.append('..')
import numpy as np

class BasicCBOW:
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size
        
        W_in = 0.01 * np.random.randn(V,H).astype('f')
        W_out = 0.01 * np.random.randn(H,V).astype('f')
        
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()
        
        layers = [self.in_layer0,
                  self.in_layer1,
                  self.out_layer
                 ]
        
        self.params, self.grads = [], []
        
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
        
        self.word_vecs = W_in   
        
    def forward(self, contexts, target):
        h0 = self.in_layer0.forward(contexts[:,0])
        h1 = self.in_layer1.forward(contexts[:,1])
        h = 0.5*(h0+h1)
        score = self.out_layer(h)
        loss = self.loss_layer(score, target)
        return loss
    
    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *=0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None

## 이제 마지막으로 이 코드들을 가지고 학습이 되도록 해보자.

In [ ]:
import sys
sys.path.append('..') 

window_size = 1
hidden_size = 5
batch_size = 3
max_epoch = 1000

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

model = SimpleCBOW(vocab_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

word_vecs = model.word_vecs
for word_id, word in id_to_word.items():
    print(word, word_vecs[word_id])

# 지금까지 구현한 CBOW모델을 개선해보자.  
-지금까지 구현한 모델은 단점이 있다.  
1.은닉층에 들어갈떄, 지금은 단어의 수가 적지만 100만 단위가 되면 연산량이 많이 소모되고 W행렬의 크기또한 커진다. 중간의 은닉층에서의 역할이 인덱싱임을 생각하고 곱하기 연산을 줄여보자.  
2.현재는 최종 출력을 softmax를 사용하지만 위와 마찬가지로 단위가 상승되면 그 연산도 만만치 않다. 따라서 새로운 방법인 네거티브 샘플링을 사용하자.  
-네거티브 샘플링은 간단하게 말해서 학습을 할 때 맞는 정답만 훈련하는게 아니라 틀린 단어도 같이 학습을 하고 나온 스코어들을 다 더한후에 로스를 계산한다.  
-여기에는 기본적으로 각각을 시그모이드 함수로 변경해주고 one vs all 느낌으로 관점을 바꾸어서 계산을 하게된다.